In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
import time

chrome_path = r"C:\chromedriver.exe"  # Update path if needed

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=Service(chrome_path), options=options)

url = "https://www.cricbuzz.com/cricket-series/3472/indian-premier-league-2021/matches"
driver.get(url)
time.sleep(3)

soup = BeautifulSoup(driver.page_source, "html.parser")
driver.quit()

matches_data = []

match_cards = soup.select("div.cb-col.cb-col-100.cb-ltst-wgt-hdr")

for card in match_cards:
    rows = card.select("div.cb-col.cb-col-100.cb-scrd-itms")

    for row in rows:
        series_name = "Indian Premier League 2021"
        year = "2021"
        series_type = "T20"

        match_info_tag = row.find("a", class_="text-hvr-underline")
        if match_info_tag:
            match_name = match_info_tag.text.strip()
            match_href = "https://www.cricbuzz.com" + match_info_tag.get("href", "")
        else:
            match_name = match_href = "Unknown"

        match_teams = row.find_all("div", class_="cb-ovr-flo cb-hmscg-tm-nm")
        match_team1 = match_teams[0].text.strip() if len(match_teams) > 0 else "Unknown"
        match_team2 = match_teams[1].text.strip() if len(match_teams) > 1 else "Unknown"

        match_status_tag = row.find("div", class_="cb-ovr-flo cb-text-complete")
        match_status = match_status_tag.text.strip() if match_status_tag else "Scheduled"

        date_venue_tag = row.find("div", class_="cb-col-100 cb-col")
        if date_venue_tag:
            date_venue_text = date_venue_tag.text.strip().split(",")
            match_datetime_start = date_venue_text[0].strip() if len(date_venue_text) > 0 else "Unknown"
            match_venue = ",".join(date_venue_text[1:]).strip() if len(date_venue_text) > 1 else "Unknown"
        else:
            match_datetime_start = match_venue = "Unknown"

        matches_data.append({
            "year": year,
            "series_type": series_type,
            "series_name": series_name,
            "match_name": match_name,
            "match_href": match_href,
            "match_team1": match_team1,
            "match_team2": match_team2,
            "match_status": match_status,
            "match_datetime_start": match_datetime_start,
            "match_venue": match_venue
        })

# Create DataFrame and save
df = pd.DataFrame(matches_data)
print(df.head())
df.to_csv("ipl_2021_matches.csv", index=False)


Empty DataFrame
Columns: []
Index: []
